In [9]:
!pip install datasets transformers huggingface_hub -q

In [10]:
from datasets import load_dataset,load_metric

In [11]:
dataset = load_dataset('csv', data_files={'train': ['train.csv'],'test': 'test.csv'})

Using custom data configuration default-ffdc1c768960782a
Reusing dataset csv (/home/.cache/huggingface/datasets/csv/default-ffdc1c768960782a/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [13]:
def preprocess_function(examples):
   return tokenizer(examples["SMS"], truncation=True)
 
tokenized_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_test = dataset['test'].map(preprocess_function, batched=True)

Loading cached processed dataset at /home/.cache/huggingface/datasets/csv/default-ffdc1c768960782a/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-6e1d229d513086c0.arrow
Loading cached processed dataset at /home/.cache/huggingface/datasets/csv/default-ffdc1c768960782a/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-9d179537d90aea12.arrow


In [14]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [16]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

In [17]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [19]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 36 not upgraded.
Need to get 3316 kB of archives.
After this operation, 11.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 git-lfs amd64 2.9.2-1 [3316 kB]
Fetched 3316 kB in 3s (1057 kB/s)               
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline

78Selecting previously unselected package git-lfs.
(Reading database ... 24576 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.9.2-1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [#

In [20]:
from transformers import TrainingArguments, Trainer
 
repo_name = "sms_spam_detection-manning"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Cloning https://huggingface.co/satish860/sms_spam_detection-manning into local empty directory.


In [21]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: SMS. If SMS are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3901
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 488


Step,Training Loss


Saving model checkpoint to sms_spam_detection-manning/checkpoint-244
Configuration saved in sms_spam_detection-manning/checkpoint-244/config.json
Model weights saved in sms_spam_detection-manning/checkpoint-244/pytorch_model.bin
tokenizer config file saved in sms_spam_detection-manning/checkpoint-244/tokenizer_config.json
Special tokens file saved in sms_spam_detection-manning/checkpoint-244/special_tokens_map.json
tokenizer config file saved in sms_spam_detection-manning/tokenizer_config.json
Special tokens file saved in sms_spam_detection-manning/special_tokens_map.json
Saving model checkpoint to sms_spam_detection-manning/checkpoint-488
Configuration saved in sms_spam_detection-manning/checkpoint-488/config.json
Model weights saved in sms_spam_detection-manning/checkpoint-488/pytorch_model.bin
tokenizer config file saved in sms_spam_detection-manning/checkpoint-488/tokenizer_config.json
Special tokens file saved in sms_spam_detection-manning/checkpoint-488/special_tokens_map.json




TrainOutput(global_step=488, training_loss=0.052631788566464284, metrics={'train_runtime': 39.4636, 'train_samples_per_second': 197.701, 'train_steps_per_second': 12.366, 'total_flos': 125931736629732.0, 'train_loss': 0.052631788566464284, 'epoch': 2.0})

In [22]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: SMS. If SMS are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1673
  Batch size = 16


{'eval_loss': 0.05117662250995636,
 'eval_accuracy': 0.9886431560071728,
 'eval_f1': 0.9573033707865168,
 'eval_runtime': 3.4027,
 'eval_samples_per_second': 491.664,
 'eval_steps_per_second': 30.858,
 'epoch': 2.0}

In [23]:
trainer.push_to_hub()

Saving model checkpoint to sms_spam_detection-manning
Configuration saved in sms_spam_detection-manning/config.json
Model weights saved in sms_spam_detection-manning/pytorch_model.bin
tokenizer config file saved in sms_spam_detection-manning/tokenizer_config.json
Special tokens file saved in sms_spam_detection-manning/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 32.0k/255M [00:00<?, ?B/s]

Upload file runs/Apr22_02-20-29_8d95c1499846/events.out.tfevents.1650594047.8d95c1499846.356.0: 100%|#########…

Upload file runs/Apr22_02-20-29_8d95c1499846/events.out.tfevents.1650594102.8d95c1499846.356.2: 100%|#########…

To https://huggingface.co/satish860/sms_spam_detection-manning
   23f7337..a582d58  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9886431560071728}, {'name': 'F1', 'type': 'f1', 'value': 0.9573033707865168}]}
To https://huggingface.co/satish860/sms_spam_detection-manning
   a582d58..72533bd  main -> main



'https://huggingface.co/satish860/sms_spam_detection-manning/commit/a582d58c2470080beccc8b5c1c18b652fb064394'